In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import matplotlib.cm as cm
import pandas as pd
import numpy as np
import time
import re
import gc
import string
from textblob import Word, TextBlob
from nltk.corpus import stopwords
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.fixes import signature

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier


# from pyspark import SparkContext
# from spark_sklearn.util import createLocalSparkSession
# from pyspark.serializers import BatchedSerializer, PickleSerializer
# from spark_sklearn import GridSearchCV
# from pyspark import broadcast
# from zodbpickle import pickle

# Setup Seaborn
sns.set(style='whitegrid', font_scale=0.8, rc={"lines.linewidth":2, 'grid.color': '.8', 'grid.linestyle':':'})
# Set up Matplotlib
matplotlib.rc('xtick', labelsize=12)     
matplotlib.rc('ytick', labelsize=12)

In [2]:
## read dataframe from story telling procedure
df_train = pd.read_csv('./data/df_train_v4.csv.gz', compression='gzip', parse_dates=['reviewTime'], low_memory=False)
df_test = pd.read_csv('./data/df_test_v4.csv.gz', compression='gzip', parse_dates=['reviewTime'], low_memory=False)

In [3]:
df_train.head(3)

,reviewerID,asin,reviewTime,categories,helpful_num,review,polarity,word_count,word_density,helpfulness
0,A3DMPV6RWRHPM3,0306813920,2005-11-30,Books,26,"Informative, but unbalanced.. This bio gave me...",0.256913,353,5.779661,5.0
1,AZYJ9TS07B02W,B000O178BY,2007-04-19,CDs & Vinyl,8,"Boogie Apocalypse. Yeow, what a disappointment...",-0.016427,189,6.021053,2.0
2,A18U7ZLAA90PNM,B004TLH6HQ,2011-06-07,Electronics,26,False advertising - Does NOT fit IPad 2. I hav...,-0.022692,154,5.148387,5.0


In [4]:
df_test.head(3)

,reviewerID,asin,reviewTime,categories,helpful_num,review,polarity,word_count,word_density,helpfulness
0,AQRVDI6DFSZ86,B001MYIXAC,2009-04-02,Movies & TV,27,"Great Film, Great Transfer, BOGUS SUBTITLES!. ...",0.173042,252,6.351779,4.0
1,A1248JRIINWRTH,B000NQ95H0,2009-10-10,Home & Kitchen,10,Terrific quality for a great price. I was a li...,0.156119,192,5.673575,5.0
2,A2F540P3L6P5CL,0547074239,2008-08-31,Books,9,In need of some serious overhaul prior to publ...,-0.007879,197,5.474747,3.0


In [5]:
df_train.shape, df_test.shape

((994036, 10), (248510, 10))

As we did in EDA project, let's combine categories 'Electronics' and 'All Electronics' into one 'Electronics'

In [6]:
## Merge All Electronics to Electronics
df_train.categories = df_train.categories.replace({'All Electronics': 'Electronics'})
df_test.categories = df_test.categories.replace({'All Electronics': 'Electronics'})

## Tfidf features

In [7]:
## prepare stopwords to remove common 1000 frequent words
text = df_train.review.str.cat(sep=" ")
stopwords_list = Counter(text.lower().split()).most_common(1000)
stopwords_list = [item[0] for item in stopwords_list]
stopwords_set = stopwords.words('english') + stopwords_list \
                + ['quot', 'Ye', 'dr', 'etc', 'mr', 'ye', 'yes', 'paul', 'oh', 'hes', 'shes',
                  'b', 'c', 'de', 'f', 'ii', 'im', 'ive', 'la', 'michael', 'ms', 'miss', 'theres',
                  'p', 'w', 'x', 'youll', 'youre', 'youd', 'cds', 'youve', 'ill', 'theyre',
                  'theyd', 'therere', 'thats']
stopwords_set = set([re.sub("[^a-zA-Z]", "", word).lower() for word in stopwords_set]) - {""}

## read in already processed clean text dataframes
X_train_review_clean = pd.read_csv('./data/X_train_review_clean.csv.gz', header=None, compression='gzip', low_memory=False)
X_test_review_clean = pd.read_csv('./data/X_test_review_clean.csv.gz', header=None, compression='gzip', low_memory=False)

## by default, pd.read_csv reads data in dataframe. Change it to pd.Series
X_train_review_clean = X_train_review_clean[0]
X_test_review_clean = X_test_review_clean[0]

In [8]:
df_train.review.head(3)

0    Informative, but unbalanced.. This bio gave me...
1    Boogie Apocalypse. Yeow, what a disappointment...
2    False advertising - Does NOT fit IPad 2. I hav...
Name: review, dtype: object

In [9]:
X_train_review_clean.head(3)

0    informative but unbalance this bio give me a p...
1    boogie apocalypse yeow what a disappointment d...
2    false advertise do not fit ipad i have a zierr...
Name: 0, dtype: object

In [12]:
## filter out stop words of English, and too frequently used words are out, 
## words used in less than 1% and more than 90% reviews excluded
## After running several times this code block, max_df: 0.55~0.90 does not change vocabulary number 
## take top 500 frequent terms 
vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,1), 
                             min_df=0.005, max_df=0.9, max_features=500, stop_words=stopwords_set)

V_train = vectorizer.fit_transform(X_train_review_clean.astype('U'))
V_test = vectorizer.transform(X_test_review_clean.astype('U'))

## dimension and type check
V_train.shape, V_train

((994036, 500), <994036x500 sparse matrix of type '<class 'numpy.float64'>'
 	with 9564638 stored elements in Compressed Sparse Row format>)

## Dimension reduction with TruncatedSVD

In [13]:
## Dimension reduction to 300
svd = TruncatedSVD(n_components=300, n_iter=7, random_state=7979)
svd.fit(V_train)

print("Reduced 300-feature data holds {:.2f} % variance of the original data".format(svd.explained_variance_ratio_.sum() * 100))

## transform arrays
V_train_new = svd.transform(V_train)
V_test_new = svd.transform(V_test)

## change to data frames
columns = ['pc'+str(i) for i in range(1, 300+1)]
V_train_new = pd.DataFrame(V_train_new, columns=columns)
V_test_new = pd.DataFrame(V_test_new, columns=columns)

## dimension check
V_train_new.shape, V_test_new.shape

Reduced 300-feature data holds 71.40 % variance of the original data


((994036, 300), (248510, 300))

## Get dummy variables for categories

In [14]:
## get dummy variabels for categories
train_cat = pd.get_dummies(df_train.categories, prefix='cat')
test_cat = pd.get_dummies(df_test.categories, prefix='cat')

## drop original review, categories columns and concatenate dummy variables
df_train = pd.concat([df_train.drop(columns=['categories']), train_cat], axis=1)
df_test = pd.concat([df_test.drop(columns=['categories']), test_cat], axis=1)

## check dimension
df_train.shape, df_test.shape

((994036, 38), (248510, 38))

## Put it all together!

In [15]:
## index reset for V_new dataframes
V_train_new.index = df_train.index
V_test_new.index = df_test.index

## Combine all data frames 
df_train = pd.concat([df_train.drop(columns=['review']), V_train_new], axis=1)
df_test = pd.concat([df_test.drop(columns=['review']), V_test_new], axis=1)

## check dimension
df_train.shape, df_test.shape

((994036, 337), (248510, 337))

In [16]:
df_train.head(3)

,reviewerID,asin,reviewTime,helpful_num,polarity,word_count,word_density,helpfulness,cat_Appliances,cat_Apps for Android,...,pc291,pc292,pc293,pc294,pc295,pc296,pc297,pc298,pc299,pc300
0,A3DMPV6RWRHPM3,0306813920,2005-11-30,26,0.256913,353,5.779661,5.0,0,0,...,0.015590,0.017478,0.010573,0.059862,0.051233,-0.004555,0.009048,-0.062087,0.054748,0.001599
1,AZYJ9TS07B02W,B000O178BY,2007-04-19,8,-0.016427,189,6.021053,2.0,0,0,...,-0.059889,0.011316,0.009066,0.057939,-0.019484,-0.015208,-0.003141,0.037956,0.041262,-0.108097
2,A18U7ZLAA90PNM,B004TLH6HQ,2011-06-07,26,-0.022692,154,5.148387,5.0,0,0,...,-0.007588,-0.003663,-0.010679,-0.002417,0.009921,0.001906,0.008329,-0.007434,-0.005938,-0.007134


## Standardization

In [17]:
## standardize numerical features
columns = df_train.columns.drop(['reviewerID', 'asin', 'reviewTime', 'helpfulness'])

## standardizing them
scaler = StandardScaler()
X_train = df_train[columns].copy()
X_test = df_test[columns].copy()
X_train[columns] = scaler.fit_transform(df_train[columns])
X_test[columns] = scaler.transform(df_test[columns])

## Split into predictors and response
y_train = df_train.helpfulness.astype(int)
y_test = df_test.helpfulness.astype(int)

## check dimension and summary statistics
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train.describe()

(994036, 333) (248510, 333) (994036,) (248510,)


,helpful_num,polarity,word_count,word_density,cat_Appliances,cat_Apps for Android,"cat_Arts, Crafts & Sewing",cat_Automotive,cat_Baby,cat_Baby Products,...,pc291,pc292,pc293,pc294,pc295,pc296,pc297,pc298,pc299,pc300
count,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,...,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05,9.940360e+05
mean,5.178760e-16,-2.166002e-16,-5.021578e-17,-3.793256e-15,-1.448696e-14,8.757220e-15,1.027800e-14,7.586651e-15,-5.933334e-15,-1.120757e-14,...,3.026219e-16,5.180992e-16,4.679954e-17,-2.088562e-17,2.880885e-16,-1.476404e-17,7.776454e-16,-2.265700e-17,3.130718e-16,-1.905817e-16
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-3.858956e-01,-6.107773e+00,-8.906790e-01,-7.708145e+00,-3.248693e-02,-2.016406e-02,-6.454477e-02,-7.705351e-02,-8.598270e-02,-2.807702e-02,...,-6.644358e+00,-6.188888e+00,-6.504232e+00,-6.626838e+00,-6.048934e+00,-7.119678e+00,-6.373493e+00,-6.540614e+00,-6.784550e+00,-6.452385e+00
25%,-2.427416e-01,-5.158984e-01,-5.810408e-01,-6.267710e-01,-3.248693e-02,-2.016406e-02,-6.454477e-02,-7.705351e-02,-8.598270e-02,-2.807702e-02,...,-5.323144e-01,-5.327463e-01,-5.069046e-01,-5.008649e-01,-5.092769e-01,-4.788405e-01,-4.910171e-01,-4.966676e-01,-4.884351e-01,-4.919479e-01
50%,-1.711645e-01,-4.528107e-04,-3.058068e-01,-4.992810e-02,-3.248693e-02,-2.016406e-02,-6.454477e-02,-7.705351e-02,-8.598270e-02,-2.807702e-02,...,-4.044235e-03,-1.560881e-03,5.647878e-03,9.869426e-04,-1.238488e-03,-2.720205e-03,2.391959e-02,-9.617627e-03,8.231765e-03,-1.890337e-02
75%,-1.011621e-02,5.287835e-01,2.145574e-01,5.737718e-01,-3.248693e-02,-2.016406e-02,-6.454477e-02,-7.705351e-02,-8.598270e-02,-2.807702e-02,...,5.098305e-01,5.175950e-01,4.920822e-01,5.112307e-01,4.852234e-01,4.881347e-01,4.884094e-01,4.719028e-01,4.680939e-01,4.490253e-01
max,1.450049e+02,4.478493e+00,2.551028e+01,1.526134e+02,3.078161e+01,4.959320e+01,1.549312e+01,1.297799e+01,1.163025e+01,3.561632e+01,...,7.861697e+00,7.501955e+00,7.736336e+00,6.750965e+00,7.614505e+00,7.589073e+00,7.803906e+00,6.582658e+00,7.929147e+00,9.044717e+00


In [18]:
pd.concat([X_train, y_train], axis=1).to_csv('./data/df_train_final.csv.gz', encoding='utf-8', index=False, compression='gzip')
pd.concat([X_test, y_test], axis=1).to_csv('./data/df_test_final.csv.gz', encoding='utf-8', index=False, compression='gzip')

In [19]:
X_train.head(3)

,helpful_num,polarity,word_count,word_density,cat_Appliances,cat_Apps for Android,"cat_Arts, Crafts & Sewing",cat_Automotive,cat_Baby,cat_Baby Products,...,pc291,pc292,pc293,pc294,pc295,pc296,pc297,pc298,pc299,pc300
0,0.079355,0.545234,0.618807,0.407406,-0.032487,-0.020164,-0.064545,-0.077054,-0.085983,-0.028077,...,0.398561,0.446251,0.276731,1.539429,1.315107,-0.119982,0.232155,-1.606384,1.425175,0.039296
1,-0.242742,-0.901591,-0.086480,0.925713,-0.032487,-0.020164,-0.064545,-0.077054,-0.085983,-0.028077,...,-1.532568,0.288389,0.238078,1.490016,-0.501852,-0.394231,-0.081980,0.977382,1.076143,-2.802619
2,0.079355,-0.934753,-0.236998,-0.948043,-0.032487,-0.020164,-0.064545,-0.077054,-0.085983,-0.028077,...,-0.194436,-0.095415,-0.268150,-0.061131,0.253660,0.046343,0.213617,-0.194900,-0.145444,-0.186946


In [20]:
X_test.head(3)

,helpful_num,polarity,word_count,word_density,cat_Appliances,cat_Apps for Android,"cat_Arts, Crafts & Sewing",cat_Automotive,cat_Baby,cat_Baby Products,...,pc291,pc292,pc293,pc294,pc295,pc296,pc297,pc298,pc299,pc300
0,0.097249,0.101297,0.184454,1.635836,-0.032487,-0.020164,-0.064545,-0.077054,-0.085983,-0.028077,...,-0.262569,0.590528,-1.830406,0.564682,0.354593,-0.967970,0.631045,0.102797,0.637230,0.033021
1,-0.206953,0.011719,-0.073578,0.179622,-0.032487,-0.020164,-0.064545,-0.077054,-0.085983,-0.028077,...,-0.718051,-0.835363,1.551426,1.299062,-1.169157,0.006471,-0.304632,-0.195367,-1.000982,0.176938
2,-0.224847,-0.856343,-0.052076,-0.247294,-0.032487,-0.020164,-0.064545,-0.077054,-0.085983,-0.028077,...,0.150765,-0.409631,-1.459845,0.892071,-1.331015,0.539980,-0.622941,-0.525938,1.174542,1.241735


In [21]:
## read dataframe from story telling procedure
df_train = pd.read_csv('./data/df_train_final.csv.gz', compression='gzip', low_memory=False)
df_test = pd.read_csv('./data/df_test_final.csv.gz', compression='gzip', low_memory=False)

## split into X, y and train and test set
X_train = df_train.drop(columns=['helpfulness'])
y_train = df_train.helpfulness
X_test = df_test.drop(columns=['helpfulness'])
y_test = df_test.helpfulness

## freeing unnecessary variables from memories
df_train = None
df_test = None
gc.collect();

## check 
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((994036, 333), (248510, 333), (994036,), (248510,))

## Machine learning frameworks
I will consider simple, somewhat simple, complex models as candidate models. That is, I will try __Logistic Regression and Gradient Boosting frameworks and evaluate them with AP (Average Precision) score__ because we have 6 class of helpfulness ($\in$ \[0, 5]) and class sizes are unbalanced (skewed downward) (Reference [this paper](https://www.biostat.wisc.edu/~page/rocpr.pdf) and [scikit-learn guide](https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py)). 

In information retrieval, precision is a measure of result relevancy, while recall is a measure of how many truly relevant results are returned. The precision-recall curve shows the tradeoff between precision and recall for different threshold. A high area under the curve represents both high recall and high precision, where high precision relates to a low false positive rate, and high recall relates to a low false negative rate. High scores for both show that the classifier is returning accurate results (high precision), as well as returning a majority of all positive results (high recall). 

<table><tr>
    <td>
        <figure>
        <img src="https://upload.wikimedia.org/wikipedia/commons/2/26/Precisionrecall.svg" style="height:600px" align='right'>
            <figcaption> (Precision and Recall - wikipedia) </figcaption>
        </figure>
    </td>
</tr></table>

Above figure is a great graphical summary of precision and recall. By definition, 
- Precision is the proportion of the data points our model says was relevant actually were relevant.
- Recall is the ability to find all relevant instances in a dataset. 
More formally,

$$ Precision = \frac{True Positives}{True Positives + False Positives}$$

$$ Recall = \frac{True Positives}{True Positives + False Negatives}$$

- Average precision (AP) summarizes a precision-recall curve. You can think of it as approximated precision-recall AUC (Area Under the Curve) score. 

$$ AP = \sum_{n}(R_{n} - R_{n-1}) P_{n}$$
Where $P_n$ and $R_n$ are the precision and recall at the $n^{th}$ threshold. Precision-recall curves are typically used in binary classification to study the output of a classifier. In order to extend the precision-recall curve and average precision to multi-class or multi-label classification, __it is necessary to binarize the output__. 

In [22]:
## Use label_binarize to be multi-label setting
Y_train = label_binarize(y_train, classes=[0, 1, 2, 3, 4, 5])
Y_test = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5] )
n_classes = Y_train.shape[1]

## setting stratifiedKFold - 3 folds
folds = 3
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 7777)

### Logistic Regression

In [23]:
## Grid Search for logistic regression
lr = OneVsRestClassifier(LogisticRegression(random_state=1004))
lr_params = {'estimator__C':[0.1, 0.5, 1, 5, 10]}

lr_grid = GridSearchCV(lr, param_grid=lr_params,
                    scoring='average_precision', 
                    cv=skf.split(X_train, y_train))

## clock running time
start = time.time()
lr_grid.fit(X_train, Y_train)
end = time.time()

print("Running time for grid search with 3-fold CV: {:.0f} (sec)".format(end-start))

Running time for grid search with 3-fold CV: 4896 (sec)


In [24]:
lr_best = lr_grid.best_estimator_
lr_score = lr_grid.best_score_
lr_params = lr_grid.best_params_

print("Best logistic regression estimator:")
print(lr_best)
print("Best logistic regression parameters:")
print(lr_params)
print("Best mean AP score for {}-fold grid search:".format(folds))
print(lr_score)

Best logistic regression estimator:
OneVsRestClassifier(estimator=LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1004, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)
Best mean AP score for 3-fold search with 5 parameter combinations:
0.4244511285111373



In [25]:
# ## AP (Average Precision) and AUC (Area Under the Curve), precision, recall, f1 scores on test data
# lr_ap = average_precision_score(Y_test, lr_best.predict_proba(X_test))
# lr_auc = roc_auc_score(Y_test, lr_best.predict_proba(X_test))
# lr_precision, lr_recall, lr_f1, _ = precision_recall_fscore_support(Y_test, lr_best.predict(X_test), average='micro')

# print("Best logistic regression estimator AP score on test:")
# print("{:.4f}".format(lr_ap))
# print("Best logistic regression estimator AUC score on test:")
# print("{:.4f}".format(lr_aud))
# print("Best logistic regression estimator Precision score on test:")
# print("{:.4f}".format(lr_precision))
# print("Best logistic regression estimator Recall score on test:")
# print("{:.4f}".format(lr_recall))
# print("Best logistic regression estimator F1 score on test:")
# print("{:.4f}".format(lr_f1))

Best logistic regression estimator AP score on test:
0.4259
Best logistic regression estimator AUC score on test:
0.7667
Best logistic regression estimator Precision score on test:
0.6688
Best logistic regression estimator Recall score on test:
0.2050
Best logistic regression estimator F1 score on test:
0.3138



In [ ]:
## AP (Average Precision) and AUC (Area Under the Curve), precision, recall, 
## f1 scores on test data for each class
y_lr_score = lr_best.predict_proba(X_test)
lr_precision = dict()
lr_recall = dict()
lr_ap = dict()
for i in range(n_classes):
    lr_precision[i], lr_recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                              y_lr_score[:, i])
    lr_ap[i] = average_precision_score(Y_test[:, i], y_lr_score[:, i])

# A "micro-average": quantifying score on all classes jointly
lr_precision["micro"], lr_recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
                                                                      y_lr_score.ravel())
lr_ap["micro"] = average_precision_score(Y_test, y_lr_score, average="micro")
lr_auc = roc_auc_score(Y_test, y_lr_score)
lr_f1 = f1_score(Y_test, lr_best.predict(X_test), average='micro')
lr_acc = accuracy_score(Y_test, lr_best.predict(X_test))

print("Best logistic regression estimator micro-averaged AP score on test data:")
print("{:.4f}".format(lr_ap["micro"]))
print("Best logistic regression estimator micro-averaged AUC score on test data:")
print("{:.4f}".format(lr_auc))
print("Best logistic regression estimator micro-averaged Precision score on test data:")
print("{:.4f}".format(lr_precision["micro"]))
print("Best logistic regression estimator micro-averaged Recall score on test data:")
print("{:.4f}".format(lr_recall["micro"]))
print("Best logistic regression estimator micro-averaged F1 score on test data:")
print("{:.4f}".format(lr_f1))
print("Best logistic regression estimator micro-averaged Accuracy score on test data:")
print("{:.4f}".format(lr_acc))

### Plot the precision-Recall curve

In [ ]:
plt.figure()
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(lr_recall['micro'], lr_precision['micro'], color='b', alpha=0.2,
         where='post')
plt.fill_between(lr_recall["micro"], lr_precision["micro"], alpha=0.2, color='b',
                 **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
    .format(lr_ap["micro"]))

## Kernel SVM (RBF kernel)

In [ ]:
## Randomized search for linear svc
svm = SVC(random_state=330)
svm_params = {
            'C':[0.001, 0.01, 0.1, 1, 10],
            'gamma':[0.001, 0.01, 0.1, 1],
             }
## number of parameter combinations that will be used for randomized search
param_comb = 10 # out of 20 (5x4) comb
svm_RS = RandomizedSearchCV(svm, param_distributions=svm_params, 
                            n_iter=param_comb, scoring='average_precision', 
                            ## run 4 jobs in parall for acceleration
                            n_jobs=4, cv=skf.split(X_train, y_train))

## clock running time
start = time.time()
svm_RS.fit(X_train, y_train)
end = time.time()

print("Running time for randomized search with 3-fold CV: {:.0f} (sec)".format(end-start))

In [ ]:
svm_best = svm_RS.best_estimator_
svm_score = svm_RS.best_score_
svm_params = svm_RS.best_params_

print("Best SVM estimator:")
print(svm_best)
print("Best SVM parameters:")
print(svm_params)
print("Best mean AP score for {}-fold search with {} parameter combinations:".format(folds, param_comb))
print(svm_score)

In [ ]:
# svm_ap = average_precision_score(Y_test, y_svm_score)
# svm_auc = roc_auc_score(Y_test, y_svm_score)
# svm_precision, svm_recall, svm_f1, _ = precision_recall_fscore_support(Y_test, svm_best.predict(X_test), average='micro')

# print("Best SVM estimator AP score on test:")
# print("{:.4f}".format(svm_ap))
# print("Best SVM estimator AUC score on test:")
# print("{:.4f}".format(svm_auc))
# print("Best SVM estimator Precision score on test:")
# print("{:.4f}".format(svm_precision))
# print("Best SVM estimator Recall score on test:")
# print("{:.4f}".format(svm_recall))
# print("Best SVM estimator F1 score on test:")
# print("{:.4f}".format(svm_f1))

In [ ]:
from sklearn.utils.fixes import signature
from sklearn.metrics import f1_score, accuracy_score
svm_acc = accuracy_score(Y_test, svm_best.predict(X_test))
svm_acc

In [ ]:
## AP (Average Precision) and AUC (Area Under the Curve), precision, recall, 
## f1 scores on test data for each class
y_svm_score = svm_best.decision_function(X_test)
svm_precision = dict()
svm_recall = dict()
svm_ap = dict()
for i in range(n_classes):
    svm_precision[i], svm_recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                                y_svm_score[:, i])
    svm_ap[i] = average_precision_score(Y_test[:, i], y_svm_score[:, i])

# A "micro-average": quantifying score on all classes jointly
svm_precision["micro"], svm_recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
                                                                        y_svm_score.ravel())
svm_ap["micro"] = average_precision_score(Y_test, y_svm_score, average="micro")
svm_auc = roc_auc_score(Y_test, y_svm_score)
svm_f1 = f1_score(Y_test, svm_best.predict(X_test), average='micro')
svm_acc = accuracy_score(Y_test, svm_best.predict(X_test))

print("Best SVM estimator micro-averaged AP score on test data:")
print("{:.4f}".format(svm_ap["micro"]))
print("Best SVM estimator micro-averaged AUC score on test data:")
print("{:.4f}".format(svm_auc))
print("Best SVM estimator micro-averaged Precision score on test data:")
print("{:.4f}".format(svm_precision["micro"]))
print("Best SVM estimator micro-averaged Recall score on test data:")
print("{:.4f}".format(svm_recall["micro"]))
print("Best SVM estimator micro-averaged F1 score on test data:")
print("{:.4f}".format(svm_f1))
print("Best SVM estimator micro-averaged Accuracy score on test data:")
print("{:.4f}".format(svm_acc))

### Plot the precision-Recall curve

In [ ]:
plt.figure()
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(svm_recall['micro'], svm_precision['micro'], color='b', alpha=0.2,
         where='post')
plt.fill_between(svm_recall["micro"], svm_precision["micro"], alpha=0.2, color='b',
                 **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
    .format(svm_ap["micro"]))

## Gradient Boosting 

In [ ]:
xgb_params = {
            'learning_rate': [0.01, 0.05, 0.1, 0.2]
            'min_child_weight': [1, 5, 10],
            'gamma': [0.5, 1, 1.5, 2, 5],
            'subsample': [0.6, 0.8, 1],
            'colsample_bytree': [0.6, 0.8, 1],
            'max_depth': [3, 4, 5]     
             }
## number of parameter that will be used for randomized search
param_comb = 180 # out of 1620 (4x3x5x3x3x3)

xgb = XGBClassifier(objective='multi:softmax', silent=True, random_state=1108)
xgb_RS = RandomizedSearchCV(xgb, param_distributions=xgb_params, 
                            n_iter=param_comb, scoring='average_precision', 
                            ## run 2 jobs in parall for acceleration
                            n_jobs=2, cv=skf.split(X_train, y_train))

## clock running time
start = time.time()
xgb_RS.fit(X_train, y_train, eval_metric='map', early_stopping_rounds=10)
end = time.time()

print("Running time for randomized search with 3-fold CV: {:.0f} (sec)".format(end-start))

In [ ]:
xgb_best = xgb_RS.best_estimator_
xgb_score = xgb_RS.best_score_
xgb_params = xgb_RS.best_params_

print("Best XGB estimator:")
print(xgb_best)
print("Best XGB parameters:")
print(xgb_params)
print("Best mean AP score for {}-fold search with {} parameter combinations:".format(folds, param_comb))
print(xgb_score)

In [ ]:
# ## AP (Average Precision) and AUC (Area Under the Curve), precision, recall, f1 scores on test data
# xgb_ap = average_precision_score(Y_test, xgb_best.predict_proba(X_test))
# xgb_auc = roc_auc_score(Y_test, xgb_best.predict_proba(X_test))
# xgb_precision, xgb_recall, xgb_f1, _ = precision_recall_fscore_support(Y_test, xgb_best.predict(X_test), average='micro')

# print("Best XGB estimator AP score on test:")
# print("{:.4f}".format(xgb_ap))
# print("Best XGB estimator AUC score on test:")
# print("{:.4f}".format(xgb_auc))
# print("Best XGB estimator Precision score on test:")
# print("{:.4f}".format(xgb_precision))
# print("Best XGB estimator Recall score on test:")
# print("{:.4f}".format(xgb_recall))
# print("Best XGB estimator F1 score on test:")
# print("{:.4f}".format(xgb_f1))

In [ ]:
## AP (Average Precision) and AUC (Area Under the Curve), precision, recall, 
## f1 scores on test data for each class
y_xgb_score = xgb_best.predict_proba(X_test)
xgb_precision = dict()
xgb_recall = dict()
xgb_ap = dict()
for i in range(n_classes):
    xgb_precision[i], xgb_recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                        y_xgb_score[:, i])
    xgb_ap[i] = average_precision_score(Y_test[:, i], y_xgb_score[:, i])

# A "micro-average": quantifying score on all classes jointly
xgb_precision["micro"], xgb_recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
                                                                        y_xgb_score.ravel())
xgb_ap["micro"] = average_precision_score(Y_test, y_xgb_score, average="micro")
xgb_auc = roc_auc_score(Y_test, y_xgb_score)
xgb_f1 = f1_score(Y_test, xgb_best.predict(X_test), average='micro')
xgb_acc = accuracy_score(Y_test, xgb_best.predict(X_test))

print("Best XGB estimator micro-averaged AP score on test data:")
print("{:.4f}".format(xgb_ap["micro"]))
print("Best XGB estimator micro-averaged AUC score on test data:")
print("{:.4f}".format(xgb_auc))
print("Best XGB estimator micro-averaged Precision score on test data:")
print("{:.4f}".format(xgb_precision["micro"]))
print("Best XGB estimator micro-averaged Recall score on test data:")
print("{:.4f}".format(xgb_recall["micro"]))
print("Best XGB estimator micro-averaged F1 score on test data:")
print("{:.4f}".format(xgb_f1))
print("Best XGB estimator micro-averaged Accuracy score on test data:")
print("{:.4f}".format(xgb_acc))

### Plot the precision-Recall curve

In [ ]:
plt.figure()
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(xgb_recall['micro'], xgb_precision['micro'], color='b', alpha=0.2,
         where='post')
plt.fill_between(xgb_recall["micro"], xgb_precision["micro"], alpha=0.2, color='b',
                 **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
    .format(xgb_ap["micro"]))